# 호텔 예약 데이터를 이용한 고객 수요 예측 프로젝트

## 프로젝트 개요
- **주제**: 호텔 예약 데이터를 이용하여 상황 가설에 따른 고객 수요 예측
- **가설 예시**: 오래 전에 예약한 사람의 예약 취소 가능성이 높다
- **목표**: 예약 취소 여부를 예측하는 머신러닝 모델 구축

## 분석 단계
1. 데이터 로드 및 기본 정보 확인
2. 탐색적 데이터 분석 (EDA)
3. 데이터 전처리
4. 피처 엔지니어링
5. 모델링 및 하이퍼파라미터 튜닝
6. 모델 평가


# 1. 라이브러리 Import 및 기본 설정


In [ ]:
# 데이터 처리 및 분석
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 통계 분석
from scipy import stats
from scipy.stats import chi2_contingency

# 머신러닝
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve


print("라이브러리 import 완료!")


ModuleNotFoundError: No module named 'imblearn'

: 

# 2. 데이터 로드 및 기본 정보 확인


In [ ]:
# 데이터 로드
df = pd.read_csv('data/hotel_bookings.csv')

print("=== 데이터셋 기본 정보 ===")
print(f"데이터셋 크기: {df.shape}")
print(f"행 개수: {df.shape[0]:,}")
print(f"열 개수: {df.shape[1]}")
print("\n=== 첫 5행 데이터 ===")
df.head()


In [ ]:
# 데이터 타입 및 기본 통계 정보
print("=== 데이터 타입 정보 ===")
print(df.dtypes)
print("\n=== 기본 통계 정보 ===")
df.describe()


In [ ]:
# 결측치 확인
print("=== 결측치 정보 ===")
missing_data = df.isnull().sum()
missing_percentage = (missing_data / len(df)) * 100
missing_info = pd.DataFrame({
    '결측치 개수': missing_data,
    '결측치 비율(%)': missing_percentage
}).sort_values('결측치 개수', ascending=False)

print(missing_info[missing_info['결측치 개수'] > 0])


In [ ]:
# 타겟 변수 분포 확인 (예약 취소 여부)
print("=== 타겟 변수 분포 (is_canceled) ===")
target_counts = df['is_canceled'].value_counts()
target_percentage = df['is_canceled'].value_counts(normalize=True) * 100

print("예약 취소 여부 분포:")
for i, (count, pct) in enumerate(zip(target_counts, target_percentage)):
    status = "취소" if target_counts.index[i] == 1 else "유지"
    print(f"{status}: {count:,}건 ({pct:.1f}%)")

# 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# 막대 그래프
target_counts.plot(kind='bar', ax=ax1, color=['skyblue', 'salmon'])
ax1.set_title('예약 취소 여부 분포')
ax1.set_xlabel('예약 상태 (0: 유지, 1: 취소)')
ax1.set_ylabel('건수')
ax1.tick_params(axis='x', rotation=0)

# 파이 차트
ax2.pie(target_counts.values, labels=['유지', '취소'], autopct='%1.1f%%', 
        colors=['skyblue', 'salmon'], startangle=90)
ax2.set_title('예약 취소 비율')

plt.tight_layout()
plt.show()


# 3. 탐색적 데이터 분석 (EDA)


In [ ]:
# 3.1 주요 가설 검증: Lead Time과 예약 취소의 관계
print("=== 가설 검증: 오래 전에 예약한 사람의 예약 취소 가능성 ===")

# Lead Time 기본 통계
print("Lead Time 기본 통계:")
print(df.groupby('is_canceled')['lead_time'].describe())

# Lead Time 구간별 취소율 분석
df['lead_time_group'] = pd.cut(df['lead_time'], 
                              bins=[0, 30, 90, 180, 365, df['lead_time'].max()],
                              labels=['0-30일', '31-90일', '91-180일', '181-365일', '365일+'])

cancel_by_leadtime = df.groupby('lead_time_group')['is_canceled'].agg(['count', 'sum', 'mean']).round(3)
cancel_by_leadtime.columns = ['전체건수', '취소건수', '취소율']
print("\nLead Time 구간별 취소율:")
print(cancel_by_leadtime)

# 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Lead Time 분포
df.boxplot(column='lead_time', by='is_canceled', ax=ax1)
ax1.set_title('예약 취소 여부별 Lead Time 분포')
ax1.set_xlabel('예약 상태 (0: 유지, 1: 취소)')
ax1.set_ylabel('Lead Time (일)')

# Lead Time 구간별 취소율
cancel_by_leadtime['취소율'].plot(kind='bar', ax=ax2, color='coral')
ax2.set_title('Lead Time 구간별 예약 취소율')
ax2.set_xlabel('Lead Time 구간')
ax2.set_ylabel('취소율')
ax2.tick_params(axis='x', rotation=45)
ax2.set_ylim(0, 1)

# 취소율 값 표시
for i, v in enumerate(cancel_by_leadtime['취소율']):
    ax2.text(i, v + 0.01, f'{v:.1%}', ha='center', va='bottom')

plt.tight_layout()
plt.show()


In [ ]:
# 3.2 주요 변수들과 예약 취소의 관계 분석

# 호텔 타입별 취소율
print("=== 호텔 타입별 취소율 ===")
hotel_cancel = df.groupby('hotel')['is_canceled'].agg(['count', 'sum', 'mean']).round(3)
hotel_cancel.columns = ['전체건수', '취소건수', '취소율']
print(hotel_cancel)

# 시장 세그먼트별 취소율
print("\n=== 시장 세그먼트별 취소율 ===")
market_cancel = df.groupby('market_segment')['is_canceled'].agg(['count', 'sum', 'mean']).round(3)
market_cancel.columns = ['전체건수', '취소건수', '취소율']
print(market_cancel.sort_values('취소율', ascending=False))

# 고객 타입별 취소율
print("\n=== 고객 타입별 취소율 ===")
customer_cancel = df.groupby('customer_type')['is_canceled'].agg(['count', 'sum', 'mean']).round(3)
customer_cancel.columns = ['전체건수', '취소건수', '취소율']
print(customer_cancel.sort_values('취소율', ascending=False))

# 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 호텔 타입별 취소율
hotel_cancel['취소율'].plot(kind='bar', ax=axes[0,0], color='lightblue')
axes[0,0].set_title('호텔 타입별 예약 취소율')
axes[0,0].set_ylabel('취소율')
axes[0,0].tick_params(axis='x', rotation=0)

# 시장 세그먼트별 취소율
market_cancel['취소율'].plot(kind='bar', ax=axes[0,1], color='lightgreen')
axes[0,1].set_title('시장 세그먼트별 예약 취소율')
axes[0,1].set_ylabel('취소율')
axes[0,1].tick_params(axis='x', rotation=45)

# 고객 타입별 취소율
customer_cancel['취소율'].plot(kind='bar', ax=axes[1,0], color='lightcoral')
axes[1,0].set_title('고객 타입별 예약 취소율')
axes[1,0].set_ylabel('취소율')
axes[1,0].tick_params(axis='x', rotation=45)

# ADR(Average Daily Rate) 분포
df.boxplot(column='adr', by='is_canceled', ax=axes[1,1])
axes[1,1].set_title('예약 취소 여부별 평균 일일 요금(ADR) 분포')
axes[1,1].set_xlabel('예약 상태 (0: 유지, 1: 취소)')
axes[1,1].set_ylabel('ADR')

plt.tight_layout()
plt.show()


In [ ]:
# 3.3 상관관계 분석
print("=== 수치형 변수들 간의 상관관계 ===")

# 수치형 변수들 선택
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"수치형 변수들: {numeric_cols}")

# 상관관계 매트릭스 계산
correlation_matrix = df[numeric_cols].corr()

# 타겟 변수와의 상관관계
target_corr = correlation_matrix['is_canceled'].sort_values(key=abs, ascending=False)
print("\n예약 취소와 상관관계가 높은 변수들:")
print(target_corr[1:11])  # 자기 자신 제외하고 상위 10개

# 상관관계 히트맵
plt.figure(figsize=(14, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f', cbar_kws={"shrink": .5})
plt.title('수치형 변수들 간의 상관관계 히트맵')
plt.tight_layout()
plt.show()


# 4. 데이터 전처리


In [ ]:
# 4.1 데이터 복사본 생성 및 기본 전처리
df_processed = df.copy()

print("=== 전처리 전 데이터 정보 ===")
print(f"데이터 크기: {df_processed.shape}")
print(f"결측치 개수: {df_processed.isnull().sum().sum()}")

# 4.1.1 결측치 처리
print("\n=== 결측치 처리 ===")

# children 컬럼의 결측치를 0으로 대체 (일반적으로 아이가 없는 경우)
if df_processed['children'].isnull().sum() > 0:
    df_processed['children'].fillna(0, inplace=True)
    print("children 결측치를 0으로 대체")

# country 결측치는 'Unknown'으로 대체
if df_processed['country'].isnull().sum() > 0:
    df_processed['country'].fillna('Unknown', inplace=True)
    print("country 결측치를 'Unknown'으로 대체")

# agent와 company 결측치는 0으로 대체 (에이전트나 회사를 통하지 않은 예약)
if df_processed['agent'].isnull().sum() > 0:
    df_processed['agent'].fillna(0, inplace=True)
    print("agent 결측치를 0으로 대체")
    
if df_processed['company'].isnull().sum() > 0:
    df_processed['company'].fillna(0, inplace=True)
    print("company 결측치를 0으로 대체")

print(f"전처리 후 결측치 개수: {df_processed.isnull().sum().sum()}")


In [ ]:
# 4.1.2 중복 데이터 확인 및 제거
print("=== 중복 데이터 처리 ===")
duplicate_count = df_processed.duplicated().sum()
print(f"중복 데이터 개수: {duplicate_count}")

if duplicate_count > 0:
    df_processed.drop_duplicates(inplace=True)
    print(f"중복 제거 후 데이터 크기: {df_processed.shape}")
else:
    print("중복 데이터가 없습니다.")

# 4.1.3 이상치 확인 및 처리
print("\n=== 이상치 확인 ===")

# ADR (Average Daily Rate) 이상치 확인
print("ADR 이상치 확인:")
adr_q1 = df_processed['adr'].quantile(0.25)
adr_q3 = df_processed['adr'].quantile(0.75)
adr_iqr = adr_q3 - adr_q1
adr_lower = adr_q1 - 1.5 * adr_iqr
adr_upper = adr_q3 + 1.5 * adr_iqr

print(f"ADR Q1: {adr_q1:.2f}, Q3: {adr_q3:.2f}")
print(f"ADR 정상 범위: {adr_lower:.2f} ~ {adr_upper:.2f}")

adr_outliers = df_processed[(df_processed['adr'] < adr_lower) | (df_processed['adr'] > adr_upper)]
print(f"ADR 이상치 개수: {len(adr_outliers)} ({len(adr_outliers)/len(df_processed)*100:.1f}%)")

# 음수 ADR 확인 (비현실적인 값)
negative_adr = df_processed[df_processed['adr'] < 0]
print(f"음수 ADR 개수: {len(negative_adr)}")

# 음수 ADR 제거
if len(negative_adr) > 0:
    df_processed = df_processed[df_processed['adr'] >= 0]
    print(f"음수 ADR 제거 후 데이터 크기: {df_processed.shape}")

# lead_time 이상치 확인
print(f"\nLead Time 통계:")
print(f"최솟값: {df_processed['lead_time'].min()}")
print(f"최댓값: {df_processed['lead_time'].max()}")
print(f"평균: {df_processed['lead_time'].mean():.1f}")
print(f"중앙값: {df_processed['lead_time'].median():.1f}")


In [ ]:
# 4.1.4 왜도(Skewness)와 첨도(Kurtosis) 분석
print("=== 왜도와 첨도 분석 ===")

numeric_cols = df_processed.select_dtypes(include=[np.number]).columns.tolist()
skew_kurt_analysis = pd.DataFrame({
    '변수명': numeric_cols,
    '왜도': [df_processed[col].skew() for col in numeric_cols],
    '첨도': [df_processed[col].kurtosis() for col in numeric_cols]
})

# 왜도와 첨도가 높은 변수들 확인
skew_kurt_analysis['왜도_절댓값'] = abs(skew_kurt_analysis['왜도'])
skew_kurt_analysis = skew_kurt_analysis.sort_values('왜도_절댓값', ascending=False)

print("변수별 왜도와 첨도:")
print(skew_kurt_analysis[['변수명', '왜도', '첨도']].head(10))

# 왜도가 높은 변수들 시각화
high_skew_vars = skew_kurt_analysis[skew_kurt_analysis['왜도_절댓값'] > 2]['변수명'].head(4).tolist()

if high_skew_vars:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.ravel()
    
    for i, var in enumerate(high_skew_vars):
        if i < 4:
            df_processed[var].hist(bins=50, ax=axes[i], alpha=0.7)
            axes[i].set_title(f'{var} 분포 (왜도: {df_processed[var].skew():.2f})')
            axes[i].set_ylabel('빈도')
    
    plt.tight_layout()
    plt.show()

print(f"\n왜도가 2 이상인 변수들: {len(skew_kurt_analysis[skew_kurt_analysis['왜도_절댓값'] > 2])}개")
print(f"첨도가 5 이상인 변수들: {len(skew_kurt_analysis[abs(skew_kurt_analysis['첨도']) > 5])}개")


# 5. 피처 엔지니어링


In [ ]:
# 5.1 새로운 피처 생성
print("=== 새로운 피처 생성 ===")

# 전체 숙박 일수
df_processed['total_nights'] = df_processed['stays_in_weekend_nights'] + df_processed['stays_in_week_nights']

# 전체 손님 수
df_processed['total_guests'] = df_processed['adults'] + df_processed['children'] + df_processed['babies']

# 1인당 평균 요금
df_processed['adr_per_person'] = df_processed['adr'] / df_processed['total_guests'].replace(0, 1)  # 0으로 나누기 방지

# 예약 변경 여부
df_processed['has_booking_changes'] = (df_processed['booking_changes'] > 0).astype(int)

# 특별 요청 여부
df_processed['has_special_requests'] = (df_processed['total_of_special_requests'] > 0).astype(int)

# 이전 취소 이력 여부
df_processed['has_previous_cancellations'] = (df_processed['previous_cancellations'] > 0).astype(int)

# 이전 예약 이력 여부
df_processed['has_previous_bookings'] = (df_processed['previous_bookings_not_canceled'] > 0).astype(int)

# Lead time 카테고리
df_processed['lead_time_category'] = pd.cut(df_processed['lead_time'], 
                                          bins=[0, 7, 30, 90, 365, df_processed['lead_time'].max()],
                                          labels=['매우짧음(0-7일)', '짧음(8-30일)', '보통(31-90일)', '김(91-365일)', '매우김(365일+)'])

# 도착 월을 계절로 분류
season_map = {
    'January': '겨울', 'February': '겨울', 'March': '봄',
    'April': '봄', 'May': '봄', 'June': '여름',
    'July': '여름', 'August': '여름', 'September': '가을',
    'October': '가을', 'November': '가을', 'December': '겨울'
}
df_processed['season'] = df_processed['arrival_date_month'].map(season_map)

# 주말 포함 여부
df_processed['includes_weekend'] = (df_processed['stays_in_weekend_nights'] > 0).astype(int)

print("생성된 새로운 피처들:")
new_features = ['total_nights', 'total_guests', 'adr_per_person', 'has_booking_changes', 
                'has_special_requests', 'has_previous_cancellations', 'has_previous_bookings',
                'lead_time_category', 'season', 'includes_weekend']
for feature in new_features:
    print(f"- {feature}")

print(f"\n전체 피처 개수: {len(df_processed.columns)}")


In [ ]:
# 5.2 범주형 변수 인코딩
print("=== 범주형 변수 인코딩 ===")

# 모델링을 위한 데이터프레임 복사
df_model = df_processed.copy()

# 범주형 변수들 확인
categorical_cols = df_model.select_dtypes(include=['object']).columns.tolist()
print(f"범주형 변수들: {categorical_cols}")

# 불필요한 컬럼 제거 (모델링에 직접적으로 사용하지 않을 컬럼들)
cols_to_drop = ['reservation_status_date', 'arrival_date_year', 'arrival_date_month', 
                'arrival_date_week_number', 'arrival_date_day_of_month', 'lead_time_group']

# 존재하는 컬럼만 제거
cols_to_drop = [col for col in cols_to_drop if col in df_model.columns]
df_model.drop(columns=cols_to_drop, inplace=True)
print(f"제거된 컬럼들: {cols_to_drop}")

# 범주형 변수들 다시 확인
categorical_cols = df_model.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"인코딩할 범주형 변수들: {categorical_cols}")

# Label Encoding을 적용할 변수들 (순서가 있는 변수들)
ordinal_cols = ['lead_time_category']

# One-Hot Encoding을 적용할 변수들 (순서가 없는 변수들)
nominal_cols = [col for col in categorical_cols if col not in ordinal_cols]

print(f"Label Encoding 적용: {ordinal_cols}")
print(f"One-Hot Encoding 적용: {nominal_cols}")

# Label Encoding
label_encoders = {}
for col in ordinal_cols:
    if col in df_model.columns:
        le = LabelEncoder()
        df_model[col] = le.fit_transform(df_model[col].astype(str))
        label_encoders[col] = le
        print(f"{col} Label Encoding 완료")

# One-Hot Encoding (카디널리티가 너무 높지 않은 변수들만)
for col in nominal_cols:
    if col in df_model.columns:
        unique_count = df_model[col].nunique()
        print(f"{col} 고유값 개수: {unique_count}")
        
        if unique_count <= 50:  # 카디널리티가 50 이하인 경우만 One-Hot Encoding
            dummies = pd.get_dummies(df_model[col], prefix=col, drop_first=True)
            df_model = pd.concat([df_model, dummies], axis=1)
            df_model.drop(columns=[col], inplace=True)
            print(f"{col} One-Hot Encoding 완료")
        else:
            # 카디널리티가 높은 경우 빈도 기반 인코딩
            freq_encoding = df_model[col].value_counts().to_dict()
            df_model[col + '_freq'] = df_model[col].map(freq_encoding)
            df_model.drop(columns=[col], inplace=True)
            print(f"{col} 빈도 기반 인코딩 완료")

print(f"\n인코딩 후 피처 개수: {len(df_model.columns)}")
print(f"인코딩 후 데이터 크기: {df_model.shape}")


In [ ]:
# 5.3 피처 스케일링
print("=== 피처 스케일링 ===")

# 타겟 변수 분리
X = df_model.drop('is_canceled', axis=1)
y = df_model['is_canceled']

print(f"피처 개수: {X.shape[1]}")
print(f"샘플 개수: {X.shape[0]}")
print(f"타겟 변수 분포:\n{y.value_counts()}")

# 수치형 변수들 확인
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
print(f"수치형 피처 개수: {len(numeric_features)}")

# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"학습 데이터 크기: {X_train.shape}")
print(f"테스트 데이터 크기: {X_test.shape}")
print(f"학습 데이터 타겟 분포:\n{y_train.value_counts()}")

# StandardScaler를 사용하여 수치형 피처들 스케일링
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# 수치형 피처들만 스케일링
if numeric_features:
    X_train_scaled[numeric_features] = scaler.fit_transform(X_train[numeric_features])
    X_test_scaled[numeric_features] = scaler.transform(X_test[numeric_features])
    print("수치형 피처들 스케일링 완료")

print(f"스케일링 후 학습 데이터 통계:")
print(X_train_scaled[numeric_features[:5]].describe().round(3))


In [ ]:
# 5.4 불균형 데이터 처리 (옵션)
print("=== 불균형 데이터 확인 및 처리 ===")

# 타겟 변수 불균형 확인
target_ratio = y_train.value_counts(normalize=True)
print("타겟 변수 비율:")
print(f"예약 유지 (0): {target_ratio[0]:.1%}")
print(f"예약 취소 (1): {target_ratio[1]:.1%}")

minority_ratio = min(target_ratio[0], target_ratio[1])
print(f"소수 클래스 비율: {minority_ratio:.1%}")

# 불균형이 심한 경우 (소수 클래스가 30% 미만) SMOTE 적용
if minority_ratio < 0.3:
    print("불균형 데이터 감지 - SMOTE 적용")
    
    # SMOTE 적용
    smote = SMOTE(random_state=42, k_neighbors=5)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)
    
    print(f"SMOTE 적용 전 학습 데이터 크기: {X_train_scaled.shape}")
    print(f"SMOTE 적용 후 학습 데이터 크기: {X_train_balanced.shape}")
    
    balanced_ratio = y_train_balanced.value_counts(normalize=True)
    print("SMOTE 적용 후 타겟 변수 비율:")
    print(f"예약 유지 (0): {balanced_ratio[0]:.1%}")
    print(f"예약 취소 (1): {balanced_ratio[1]:.1%}")
    
    # 균형 맞춘 데이터를 기본으로 사용
    X_train_final = X_train_balanced
    y_train_final = y_train_balanced
    
else:
    print("데이터 균형이 적절함 - 원본 데이터 사용")
    X_train_final = X_train_scaled
    y_train_final = y_train

print(f"최종 학습 데이터 크기: {X_train_final.shape}")
print(f"최종 테스트 데이터 크기: {X_test_scaled.shape}")


# 6. 모델링 및 하이퍼파라미터 튜닝


In [ ]:
# 6.1 기본 모델들 성능 비교
print("=== 기본 모델들 성능 비교 ===")

# 여러 모델들 정의
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42, n_estimators=100),
    'SVM': SVC(random_state=42, probability=True)
}

# 교차 검증을 위한 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 모델별 성능 저장
model_scores = {}

print("교차 검증 진행 중...")
for name, model in models.items():
    print(f"\n{name} 학습 중...")
    
    # 교차 검증
    cv_scores = cross_val_score(model, X_train_final, y_train_final, cv=cv, scoring='roc_auc', n_jobs=-1)
    
    model_scores[name] = {
        'CV Mean': cv_scores.mean(),
        'CV Std': cv_scores.std(),
        'CV Scores': cv_scores
    }
    
    print(f"{name} - AUC: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# 결과 정리
results_df = pd.DataFrame({
    'Model': list(model_scores.keys()),
    'CV_Mean_AUC': [scores['CV Mean'] for scores in model_scores.values()],
    'CV_Std_AUC': [scores['CV Std'] for scores in model_scores.values()]
}).sort_values('CV_Mean_AUC', ascending=False)

print("\n=== 모델 성능 순위 ===")
print(results_df)

# 시각화
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.bar(results_df['Model'], results_df['CV_Mean_AUC'], yerr=results_df['CV_Std_AUC'], capsize=5)
plt.title('모델별 교차검증 AUC 점수')
plt.ylabel('AUC Score')
plt.xticks(rotation=45)
plt.ylim(0.8, 1.0)

# 박스플롯
plt.subplot(1, 2, 2)
cv_data = [model_scores[name]['CV Scores'] for name in results_df['Model']]
plt.boxplot(cv_data, labels=results_df['Model'])
plt.title('모델별 교차검증 AUC 분포')
plt.ylabel('AUC Score')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# 6.2 최고 성능 모델 하이퍼파라미터 튜닝
print("=== 하이퍼파라미터 튜닝 ===")

# 가장 성능이 좋은 모델 선택
best_model_name = results_df.iloc[0]['Model']
print(f"최고 성능 모델: {best_model_name}")

# Random Forest 하이퍼파라미터 튜닝 (일반적으로 좋은 성능을 보이는 모델)
if 'Random Forest' in best_model_name or True:  # 예시로 Random Forest 튜닝
    print("Random Forest 하이퍼파라미터 튜닝 진행...")
    
    # 하이퍼파라미터 그리드 정의
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'max_features': ['sqrt', 'log2']
    }
    
    # GridSearchCV 설정
    rf = RandomForestClassifier(random_state=42)
    grid_search = GridSearchCV(
        rf, param_grid, cv=3, scoring='roc_auc', 
        n_jobs=-1, verbose=1, return_train_score=True
    )
    
    # 하이퍼파라미터 튜닝 실행
    print("GridSearch 실행 중... (시간이 다소 걸릴 수 있습니다)")
    grid_search.fit(X_train_final, y_train_final)
    
    # 최적 파라미터 출력
    print(f"\n최적 파라미터: {grid_search.best_params_}")
    print(f"최적 교차검증 AUC: {grid_search.best_score_:.4f}")
    
    # 최적 모델 저장
    best_model = grid_search.best_estimator_
    
else:
    # 다른 모델의 경우 기본 파라미터 사용
    if 'Logistic' in best_model_name:
        best_model = LogisticRegression(random_state=42, max_iter=1000)
    elif 'Gradient' in best_model_name:
        best_model = GradientBoostingClassifier(random_state=42, n_estimators=200)
    else:
        best_model = RandomForestClassifier(random_state=42, n_estimators=200)
    
    best_model.fit(X_train_final, y_train_final)

print("하이퍼파라미터 튜닝 완료!")


# 7. 모델 평가


In [ ]:
# 7.1 테스트 데이터 예측 및 성능 평가
print("=== 최종 모델 성능 평가 ===")

# 테스트 데이터 예측
y_pred = best_model.predict(X_test_scaled)
y_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1]

# 성능 지표 계산
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

print("=== 성능 지표 ===")
print(f"정확도 (Accuracy): {accuracy:.4f}")
print(f"정밀도 (Precision): {precision:.4f}")
print(f"재현율 (Recall): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC-ROC: {auc:.4f}")

# 혼동 행렬
print("\n=== 혼동 행렬 ===")
cm = confusion_matrix(y_test, y_pred)
print(cm)

# 분류 리포트
print("\n=== 분류 리포트 ===")
print(classification_report(y_test, y_pred, target_names=['예약유지', '예약취소']))


In [ ]:
# 7.2 시각화를 통한 성능 분석
print("=== 성능 시각화 ===")

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. 혼동 행렬 히트맵
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0,0],
            xticklabels=['예약유지', '예약취소'], yticklabels=['예약유지', '예약취소'])
axes[0,0].set_title('혼동 행렬 (Confusion Matrix)')
axes[0,0].set_ylabel('실제값')
axes[0,0].set_xlabel('예측값')

# 2. ROC 곡선
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
axes[0,1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc:.4f})')
axes[0,1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[0,1].set_xlim([0.0, 1.0])
axes[0,1].set_ylim([0.0, 1.05])
axes[0,1].set_xlabel('False Positive Rate')
axes[0,1].set_ylabel('True Positive Rate')
axes[0,1].set_title('ROC 곡선')
axes[0,1].legend(loc="lower right")

# 3. 예측 확률 분포
axes[1,0].hist(y_pred_proba[y_test==0], bins=50, alpha=0.7, label='예약유지', color='skyblue')
axes[1,0].hist(y_pred_proba[y_test==1], bins=50, alpha=0.7, label='예약취소', color='salmon')
axes[1,0].set_xlabel('예측 확률')
axes[1,0].set_ylabel('빈도')
axes[1,0].set_title('클래스별 예측 확률 분포')
axes[1,0].legend()

# 4. 성능 지표 바 차트
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
values = [accuracy, precision, recall, f1, auc]
bars = axes[1,1].bar(metrics, values, color=['lightblue', 'lightgreen', 'lightcoral', 'lightyellow', 'lightpink'])
axes[1,1].set_title('성능 지표 요약')
axes[1,1].set_ylabel('점수')
axes[1,1].set_ylim(0, 1)

# 바 위에 값 표시
for bar, value in zip(bars, values):
    axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                   f'{value:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()


In [ ]:
# 7.3 피처 중요도 분석
print("=== 피처 중요도 분석 ===")

# Random Forest의 경우 피처 중요도 확인 가능
if hasattr(best_model, 'feature_importances_'):
    # 피처 중요도 추출
    feature_importance = pd.DataFrame({
        'feature': X_train_final.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("상위 20개 중요 피처:")
    print(feature_importance.head(20))
    
    # 피처 중요도 시각화
    plt.figure(figsize=(12, 8))
    top_features = feature_importance.head(15)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('중요도')
    plt.title('상위 15개 피처 중요도')
    plt.gca().invert_yaxis()
    
    # 중요도 값 표시
    for i, v in enumerate(top_features['importance']):
        plt.text(v + 0.001, i, f'{v:.3f}', va='center')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("현재 모델은 피처 중요도를 제공하지 않습니다.")

# 7.4 Overfitting/Underfitting 검증
print("\n=== Overfitting/Underfitting 검증 ===")

# 학습 데이터에 대한 예측
y_train_pred = best_model.predict(X_train_final)
y_train_pred_proba = best_model.predict_proba(X_train_final)[:, 1]

# 학습 데이터 성능
train_accuracy = accuracy_score(y_train_final, y_train_pred)
train_auc = roc_auc_score(y_train_final, y_train_pred_proba)

print(f"학습 데이터 정확도: {train_accuracy:.4f}")
print(f"테스트 데이터 정확도: {accuracy:.4f}")
print(f"정확도 차이: {train_accuracy - accuracy:.4f}")

print(f"\n학습 데이터 AUC: {train_auc:.4f}")
print(f"테스트 데이터 AUC: {auc:.4f}")
print(f"AUC 차이: {train_auc - auc:.4f}")

# 판정
if train_auc - auc > 0.05:
    print("\n⚠️ Overfitting 의심: 학습 데이터와 테스트 데이터 성능 차이가 큽니다.")
elif auc < 0.7:
    print("\n⚠️ Underfitting 의심: 전체적인 성능이 낮습니다.")
else:
    print("\n✅ 적절한 학습: 과적합이나 과소적합 없이 잘 학습되었습니다.")
